In [ ]:
from google_maps_geocoder.geocoder import GoogleGeocoder
import pandas as pd

# Initialize the GoogleGeocoder
api_key = "Your_Key"
geocoder = GoogleGeocoder(api_key)

# Load the dataset
input_data = pd.read_csv('./example_address.csv')

# Clean and prepare the data
destinations, needs_geocoding = geocoder.cleanup_pd(input_data)

# Perform geocoding
if needs_geocoding:
    final_dest_df = geocoder.geocode_addresses(destinations, needs_geocoding)
    final_dest_df.to_csv('geocoded_results.csv', index=False)
else:
    print("Data already contains coordinates.")


Google Geocoder API connection successful!
